# Custom layers

#### https://www.tensorflow.org/tutorials/customization/custom_layers

We recommend using `tf.keras` as a high-level API for building neural networks. That said, most TensorFlow APIs are usable with eager execution.

In [1]:
import tensorflow as tf

## Layers: common sets of useful operations

Most of the time when writing code for machine learning models you want to operate at a higher level of abstraction than individual operations and manipulation of individual variables.

Many machine learning models are expressible as the composition and stacking of relatively simple layers, and TensorFlow provides both a set of many common layers as a well as easy ways for you to write your own application-specific layers either from scratch or as the composition of existing layers.

TensorFlow includes the full [Keras](https://keras.io) API in the tf.keras package, and the Keras layers are very useful when building your own models.


In [2]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))


The full list of pre-existing layers can be seen in [the documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers). It includes Dense (a fully-connected layer),
Conv2D, LSTM, BatchNormalization, Dropout, and many others.

In [3]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))

<tf.Tensor: id=29, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [4]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.59311146, -0.32651642, -0.52748215, -0.475437  ,  0.12467593,
          0.6103397 , -0.01012594, -0.568373  ,  0.5189112 ,  0.46921295],
        [ 0.53076   ,  0.41632932, -0.42799526, -0.4215513 , -0.00329447,
          0.15477318,  0.08349466,  0.10478008, -0.20283553,  0.01457077],
        [-0.22807899,  0.5134519 ,  0.2177962 , -0.6099257 ,  0.5495067 ,
         -0.4692222 ,  0.5828417 , -0.59604716,  0.10896689,  0.13313699],
        [ 0.15713328,  0.17664695, -0.5829495 , -0.54546404,  0.53877455,
          0.378614  , -0.3825729 ,  0.5860111 , -0.09358472, -0.21719715],
        [-0.02405089,  0.03644061, -0.36832374,  0.55199546,  0.26124507,
          0.29907686, -0.38121065, -0.5986494 , -0.16485325,  0.3104943 ]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [5]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.59311146, -0.32651642, -0.52748215, -0.475437  ,  0.12467593,
          0.6103397 , -0.01012594, -0.568373  ,  0.5189112 ,  0.46921295],
        [ 0.53076   ,  0.41632932, -0.42799526, -0.4215513 , -0.00329447,
          0.15477318,  0.08349466,  0.10478008, -0.20283553,  0.01457077],
        [-0.22807899,  0.5134519 ,  0.2177962 , -0.6099257 ,  0.5495067 ,
         -0.4692222 ,  0.5828417 , -0.59604716,  0.10896689,  0.13313699],
        [ 0.15713328,  0.17664695, -0.5829495 , -0.54546404,  0.53877455,
          0.378614  , -0.3825729 ,  0.5860111 , -0.09358472, -0.21719715],
        [-0.02405089,  0.03644061, -0.36832374,  0.55199546,  0.26124507,
          0.29907686, -0.38121065, -0.5986494 , -0.16485325,  0.3104943 ]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

## Implementing custom layers
The best way to implement your own layer is extending the tf.keras.Layer class and implementing:
  *  `__init__` , where you can do all input-independent initialization
  * `build`, where you know the shapes of the input tensors and can do the rest of the initialization
  * `call`, where you do the forward computation

Note that you don't have to wait until `build` is called to create your variables, you can also create them in `__init__`. However, the advantage of creating them in `build` is that it enables late variable creation based on the shape of the inputs the layer will operate on. On the other hand, creating variables in `__init__` would mean that shapes required to create the variables will need to be explicitly specified.

In [6]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self,num_outputs,**kwargs):
        self.num_outputs = num_outputs
        super(MyDenseLayer,self).__init__(**kwargs)
        
    def build(self,input_shape):
        self.kernel = self.add_weight("kernel",
                                      shape=[int(input_shape[-1]),self.num_outputs],
                                      initializer='uniform',
                                      trainable = True,
                                     )
        super(MyDenseLayer,self).build(input_shape)
        
    def call(self,input):
        return tf.matmul(input,self.kernel)

In [7]:
layer = MyDenseLayer(10)

In [8]:
_=layer(tf.zeros([10,5]))

In [9]:
print([var.name for var in layer.trainable_variables])

['my_dense_layer/kernel:0']


In [10]:
layer.kernel

<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[ 0.02050472,  0.04900212, -0.01499101,  0.02627348, -0.03491217,
        -0.00089754, -0.00769501,  0.03247441,  0.03302716, -0.01266496],
       [ 0.0496828 , -0.04515023,  0.04276292, -0.00645115, -0.00474124,
        -0.03332938,  0.01584072, -0.03521061,  0.01893982,  0.03331249],
       [-0.03992343,  0.00043442, -0.03016297, -0.04409136,  0.01814042,
        -0.04136665,  0.02406038,  0.03425341, -0.00853823, -0.04479026],
       [ 0.03256286,  0.00241245, -0.03858845,  0.03245002, -0.0368232 ,
        -0.0462988 , -0.00756212,  0.03956007, -0.02715611,  0.02591154],
       [-0.01954622,  0.04612731,  0.02505844,  0.02933441, -0.04980351,
         0.00441974,  0.04500898,  0.03237159, -0.04758978,  0.01673157]],
      dtype=float32)>

## Models: Composing layers

Many interesting layer-like things in machine learning models are implemented by composing existing layers. For example, each residual block in a resnet is a composition of convolutions, batch normalizations, and a shortcut. Layers can be nested inside other layers.

Typically you inherit from `keras.Model` when you need the model methods like: `Model.fit`,`Model.evaluate`, and `Model.save` (see [Custom Keras layers and models](../../guide/keras/custom_layers_and_models.ipynb) for details).

One other feature provided by `keras.Model` (instead of `keras.layers.Layer`) is that in addition to tracking variables, a `keras.Model` also tracks its internal layers, making them easier to inspect.

For example here is a ResNet block:

In [11]:
class ResnetIdentityBlock(tf.keras.Model):
    
    def __init__(self, kernel_size, filters):
        
        super(ResnetIdentityBlock, self).__init__(name='')
        filters1, filters2, filters3 = filters

        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        self.bn2a = tf.keras.layers.BatchNormalization()

        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()

        self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2c(x)
        x = self.bn2c(x, training=training)

        x += input_tensor
        return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])


In [12]:
_ = block(tf.zeros([1, 2, 3, 3])) 

In [13]:
_

<tf.Tensor: id=290, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [14]:
block.layers

In [15]:
len(block.variables)

18

In [17]:
block.summary()

Model: "resnet_identity_block"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  4         
_________________________________________________________________
batch_normalization (BatchNo multiple                  4         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_1 (Batch multiple                  8         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  9         
_________________________________________________________________
batch_normalization_2 (Batch multiple                  12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
________________________________________________

Much of the time, however, models which compose many layers simply call one layer after the other. This can be done in very little code using `tf.keras.Sequential`:

In [18]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: id=741, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [19]:
my_seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, None, None, 1)     4         
_________________________________________________________________
batch_normalization_3 (Batch (None, None, None, 1)     4         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 2)     4         
_________________________________________________________________
batch_normalization_4 (Batch (None, None, None, 2)     8         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, None, None, 3)     9         
_________________________________________________________________
batch_normalization_5 (Batch (None, None, None, 3)     12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
___________________________________________________________